# 추론 엔진 테스트

이 노트북은 독립 추론 엔진과 CLI 도구를 테스트합니다.

In [ ]:
import sys
from pathlib import Path
import pandas as pd

# 상위 디렉토리의 code를 import 경로에 추가
sys.path.append(str(Path.cwd().parent / 'code'))

from core.inference import InferenceEngine, InferenceConfig, create_inference_engine

## 1. 테스트 데이터 준비

In [ ]:
# 테스트용 대화 데이터
test_dialogues = [
    "#Person1#: 안녕하세요. 오늘 날씨가 정말 좋네요. #Person2#: 네, 정말 좋아요. 산책하기 딱 좋은 날씨입니다. #Person1#: 점심은 드셨나요? #Person2#: 아직이요. 뭐 먹을지 고민 중이에요.",
    "#Person1#: 프로젝트 진행 상황은 어떤가요? #Person2#: 순조롭게 진행되고 있습니다. 다음 주까지는 완료할 수 있을 것 같아요. #Person1#: 좋습니다. 문제가 있으면 바로 알려주세요.",
    "#Person1#: 주말에 영화 보러 갈래요? #Person2#: 좋아요! 무슨 영화 볼까요? #Person1#: 액션 영화는 어때요? #Person2#: 좋습니다. 시간은 언제가 좋으세요?"
]

# 테스트 DataFrame 생성
test_df = pd.DataFrame({
    'fname': [f'test_{i}.txt' for i in range(len(test_dialogues))],
    'dialogue': test_dialogues
})

print("테스트 데이터:")
test_df

## 2. 추론 엔진 초기화

In [ ]:
# 추론 설정
config = InferenceConfig(
    model_path='gogamza/kobart-summarization',  # HuggingFace Hub 모델
    batch_size=2,
    max_source_length=512,
    max_target_length=128,
    num_beams=3,
    device=None  # 자동 감지
)

print("추론 설정:")
print(f"- 모델: {config.model_path}")
print(f"- 배치 크기: {config.batch_size}")
print(f"- 디바이스: {config.device or '자동 감지'}")

In [ ]:
# 추론 엔진 생성
print("추론 엔진 초기화 중...")
engine = InferenceEngine(config)

print(f"\n초기화 완료!")
print(f"- 사용 디바이스: {engine.device}")
print(f"- 모델 타입: {engine.model_type}")
print(f"- 실제 배치 크기: {engine.config.batch_size}")

## 3. 단일 추론 테스트

In [ ]:
# 단일 대화 추론
single_dialogue = test_dialogues[0]
print("입력 대화:")
print(single_dialogue)
print("\n" + "="*50 + "\n")

# 추론 실행
summary = engine.predict_single(single_dialogue)
print("생성된 요약:")
print(summary)

## 4. 배치 추론 테스트

In [ ]:
# 배치 추론
print(f"{len(test_dialogues)}개 대화에 대한 배치 추론...")
summaries = engine.predict_batch(test_dialogues, show_progress=True)

print("\n결과:")
for i, (dialogue, summary) in enumerate(zip(test_dialogues, summaries)):
    print(f"\n[{i+1}]")
    print(f"대화: {dialogue[:50]}...")
    print(f"요약: {summary}")

## 5. DataFrame 추론 테스트

In [ ]:
# DataFrame 추론
result_df = engine.predict_from_dataframe(
    test_df,
    dialogue_column='dialogue',
    output_column='summary',
    show_progress=True
)

print("\n결과 DataFrame:")
result_df

## 6. 제출 파일 저장 테스트

In [ ]:
# 제출 형식으로 저장
output_path = "test_submission.csv"
engine.save_submission(result_df, output_path)

# 저장된 파일 확인
saved_df = pd.read_csv(output_path)
print("저장된 제출 파일:")
print(saved_df)

# 파일 삭제
import os
os.remove(output_path)
print("\n테스트 파일 삭제 완료")

## 7. CLI 도구 테스트

In [ ]:
# 테스트 데이터 저장
test_input_path = "test_input.csv"
test_df.to_csv(test_input_path, index=False)
print(f"테스트 입력 파일 생성: {test_input_path}")

In [ ]:
# CLI 도구 도움말 확인
!cd ../code && python run_inference.py --help

In [ ]:
# CLI 도구 실행 테스트 (작은 배치 크기로)
!cd ../code && python run_inference.py \
    --model_path gogamza/kobart-summarization \
    --input_file ../notebooks/test_input.csv \
    --output_file ../notebooks/test_cli_output.csv \
    --batch_size 1 \
    --max_source_length 256 \
    --max_target_length 64 \
    --num_beams 2 \
    --verbose

In [ ]:
# CLI 결과 확인
if Path('test_cli_output.csv').exists():
    cli_result = pd.read_csv('test_cli_output.csv')
    print("CLI 도구 결과:")
    print(cli_result)
    
    # 테스트 파일 정리
    os.remove('test_input.csv')
    os.remove('test_cli_output.csv')
    print("\n테스트 파일 정리 완료")

## 8. 성능 측정

In [ ]:
import time

# 추론 속도 측정
def measure_inference_speed(engine, dialogues, batch_size):
    """추론 속도 측정"""
    engine.config.batch_size = batch_size
    
    start_time = time.time()
    summaries = engine.predict_batch(dialogues, show_progress=False)
    end_time = time.time()
    
    total_time = end_time - start_time
    per_sample_time = total_time / len(dialogues)
    
    return {
        'batch_size': batch_size,
        'total_time': total_time,
        'per_sample_time': per_sample_time,
        'samples_per_second': len(dialogues) / total_time
    }

# 다양한 배치 크기로 테스트
print("추론 속도 측정:")
print(f"테스트 샘플 수: {len(test_dialogues)}")
print("\n배치크기 | 전체시간(초) | 샘플당시간(초) | 초당샘플수")
print("-" * 60)

for batch_size in [1, 2, 4]:
    try:
        result = measure_inference_speed(engine, test_dialogues, batch_size)
        print(f"{result['batch_size']:^9} | {result['total_time']:^12.2f} | "
              f"{result['per_sample_time']:^14.2f} | {result['samples_per_second']:^10.2f}")
    except Exception as e:
        print(f"{batch_size:^9} | 오류: {str(e)[:30]}...")

## 9. 헬퍼 함수 테스트

In [ ]:
# create_inference_engine 헬퍼 함수 테스트
quick_engine = create_inference_engine(
    model_path='gogamza/kobart-summarization',
    batch_size=4,
    fp16=False,
    max_source_length=256
)

# __call__ 메서드 테스트 (다양한 입력 형식)
print("1. 문자열 입력:")
result1 = quick_engine(test_dialogues[0])
print(f"   결과: {result1}")

print("\n2. 리스트 입력:")
result2 = quick_engine(test_dialogues[:2], show_progress=False)
print(f"   결과 수: {len(result2)}")

print("\n3. DataFrame 입력:")
result3 = quick_engine(test_df.head(2), show_progress=False)
print(f"   결과 shape: {result3.shape}")

## 결론

독립 추론 엔진이 성공적으로 구현되었습니다:

1. **유연한 입력 처리**: 단일 텍스트, 리스트, DataFrame 모두 지원
2. **디바이스 자동 감지**: MPS/CUDA/CPU를 자동으로 감지하고 최적화
3. **배치 처리**: 효율적인 배치 추론으로 대용량 데이터 처리 가능
4. **CLI 도구**: 명령행에서 쉽게 사용 가능한 독립 실행 도구
5. **대회 제출 형식**: fname, summary 형식의 CSV 자동 생성

이제 학습된 모델을 사용하여 대회 테스트 데이터에 대한 추론을 쉽게 수행할 수 있습니다!